In [6]:
%load_ext autoreload
%autoreload 2
from napari.qt.threading import thread_worker
import control
import napari
import gui
import time
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
config = {
    "camera": {
        "device": "zyla_camera"
    },
    "objective": {
        "device": "ti_objective",
        "states": ["a", "b", "c", "d", "e", "f"]
    },
    "light": {
        "device": "sola_light",
        "port": "COM4"
    },
    "up_filter": {
        "device": "ti_filter1",
        "states": ["mcherry", "egfp", "open1", "cy5", "brightfield", "open2"]
    },
    "down_filter": {
        "device": "ti_filter2"
    },
    "stage": {
        "device": "asi_stage",
        "port": "COM5"
    },
    "focus": {
        "device": "ti_focus"
    },
    "lightpath": {
        "device": "ti_lightpath"
    },
    "uv_filter": {
        "device": "lambda_filter1",
        "port": "COM3",
        "states": ["uv", "open1", "filter-2", "315nm", "filter-4", "340nm", "filter-6", "open2", "filter-8", "brightfield"],
    },
    "right_filter": {
        "device": "lambda_filter2",
        "port": "COM3",
        "states": ["open", "435", "474", "536", "546", "572", "620", "630", "650", "780"],
    },
    "laser": {
        "device": "lambda_shutter1",
        "port": "COM3"
    },
    "uv_shutter": {
        "device": "lambda_shutter2",
        "port": "COM3"
    },
}
c = control.load(config)
c._core.setTimeoutMs(100000)
c._core.setCameraDevice("camera")
c._core.setExposure(3)

In [13]:
viewer, worker = gui.live(c)

In [14]:
top_left = c.xy

In [15]:
bottom_right = c.xy

In [16]:
top_channels = ["open1", "open2"]
right_channels = ["572", "620"]

In [17]:
overlap_pixels = 100
um_per_pixel = 1.6137
img = c.snap()
img_height_pixels = img.shape[0]
img_width_pixels = img.shape[1]
img_height_um = um_per_pixel * img.shape[0]
img_width_um = um_per_pixel * img.shape[1]
overlap_um = overlap_pixels * um_per_pixel
delta_x = img_width_um - overlap_um
delta_y = img_height_um - overlap_um
xs = np.arange(top_left[0], bottom_right[0], delta_x)
ys = np.arange(top_left[1], bottom_right[1], delta_y)

canvas = np.zeros((len(top_channels) + len(right_channels),
                (img_height_pixels - overlap_pixels) * len(ys) + overlap_pixels,
                (img_width_pixels - overlap_pixels) * len(xs) + overlap_pixels), dtype=np.uint16)
viewer, layers = napari.imshow(canvas, channel_axis=0, contrast_limits=[0, 65535], name=top_channels + right_channels)
def update_canvas(d):
    d, i, j, c = d[0], d[1], d[2], d[3]
    row = i * (img_height_pixels - overlap_pixels)
    col = j * (img_width_pixels - overlap_pixels)
    viewer.layers[c].data[row:row + img_height_pixels, col:col + img_width_pixels] = d
    viewer.layers[c].refresh()

@thread_worker(connect={"yielded": update_canvas})
def tile_images():
    c.uv_filter = "uv"
    for i, y in enumerate(ys):
        for j, x in enumerate(xs):
            c.xy = (x, y)
            c._core.waitForDevice("stage")
            c.right_filter = "open"
            c.uv_shutter = "closed"
            c.light = 100
            c._core.setExposure(3)
            time.sleep(3)
            for channel in top_channels:
                c.up_filter = channel
                time.sleep(1)
                yield (c.snap(), i, j, channel)
                c._core.waitForDevice("camera")
                time.sleep(1)

            c.up_filter = "open1"
            c.uv_shutter = "open"
            c._core.setExposure(3)
            for channel in right_channels:
                c.right_filter = channel
                time.sleep(1)
                yield (c.snap(), i, j, channel)
                c._core.waitForDevice("camera")
                time.sleep(1)
            time.sleep(1)

In [18]:
worker2 = tile_images()

In [19]:
worker2.quit()

In [11]:
c.close()